Simple tooling to get data, edit data, query data and visualize data.
Future: Enrich Data with ML

In [1]:
##pixdust speaks spark
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime
import numpy as np
import qgrid
import pixiedust
##import pixiedust_node
randn = np.random.randn

Pixiedust database opened successfully


Get Data

In [2]:
start = datetime(2018, 4, 1)
end = datetime(2018, 4, 24)
symbols = ['F','AAPL']
##stocks = web.DataReader('F', 'iex', start, end)
##stocks = web.DataReader(symbols, 'morningstar', start, end)
##for i in symbols:
##    web.DataReader(i,'morningstar',start, end).to_csv('stocks/'+i+'.csv') 

In [3]:

import glob
filenames = glob.glob("stocks/*.csv")
stocks = pd.concat( [ pd.read_csv(f) for f in filenames ] )
stocks['iDate'] = pd.to_datetime(stocks['Date'])
stocks.describe()
##display(stocks)

,Close,High,Low,Open,Volume
count,32.000000,32.000000,32.000000,32.000000,3.200000e+01
mean,91.488438,92.410731,90.690159,91.519997,3.503153e+07
std,81.602701,82.403780,80.933954,81.615019,9.713422e+06
min,10.820000,11.030000,10.700000,10.840000,2.075454e+07
25%,11.302500,11.380000,11.222500,11.325000,2.872026e+07
50%,88.345000,89.235000,87.750000,88.190000,3.378891e+07
75%,172.530000,173.942400,171.572500,172.317500,4.279246e+07
max,178.240000,178.936500,176.880000,177.810000,6.549114e+07


In [4]:
display(stocks)

Symbol,Date,Close,High,Low,Open,Volume,iDate
F,2018-04-02,10.86,11.09,10.7,11.06,43598094,2018-04-02 00:00:00
F,2018-04-03,11.15,11.18,10.9,11.01,42523917,2018-04-03 00:00:00
F,2018-04-04,11.33,11.36,10.8,10.87,44834639,2018-04-04 00:00:00
F,2018-04-05,11.35,11.38,11.23,11.34,28755606,2018-04-05 00:00:00
F,2018-04-06,11.18,11.35,11.07,11.28,31920686,2018-04-06 00:00:00
F,2018-04-09,11.25,11.38,11.2,11.25,36462658,2018-04-09 00:00:00
F,2018-04-10,11.45,11.55,11.41,11.44,44005015,2018-04-10 00:00:00
F,2018-04-11,11.43,11.49,11.3,11.38,32979038,2018-04-11 00:00:00
F,2018-04-12,11.31,11.54,11.25,11.5,47133732,2018-04-12 00:00:00
F,2018-04-13,11.28,11.39,11.23,11.35,34598783,2018-04-13 00:00:00


In [5]:
from pyspark.ml.regression import LinearRegression  
from pyspark.ml.feature import VectorAssembler  
from pyspark.ml.feature import StandardScaler  
stockssdf = spark.createDataFrame(stocks)

stockssdf = stockssdf \
    .withColumn('label',stockssdf.Open) 
    
##features = ["Symbol", "Date"]  
features = ["Symbol", "Date"]  
label = ["label"]  

lr_data = stockssdf.select('label',*features)  
(training, test) = lr_data.randomSplit([.7, .3])

In [6]:
lr_data.show()

+------+------+----------+
| label|Symbol|      Date|
+------+------+----------+
| 11.06|     F|2018-04-02|
| 11.01|     F|2018-04-03|
| 10.87|     F|2018-04-04|
| 11.34|     F|2018-04-05|
| 11.28|     F|2018-04-06|
| 11.25|     F|2018-04-09|
| 11.44|     F|2018-04-10|
| 11.38|     F|2018-04-11|
|  11.5|     F|2018-04-12|
| 11.35|     F|2018-04-13|
| 11.36|     F|2018-04-16|
| 11.43|     F|2018-04-17|
| 11.38|     F|2018-04-18|
| 11.18|     F|2018-04-19|
| 11.01|     F|2018-04-20|
| 10.84|     F|2018-04-23|
|167.88|  AAPL|2018-04-02|
|167.64|  AAPL|2018-04-03|
|164.88|  AAPL|2018-04-04|
|172.58|  AAPL|2018-04-05|
+------+------+----------+
only showing top 20 rows



In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.clustering import KMeans
from pyspark.ml.classification import NaiveBayes ##, NaiveBayesModel
##from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import DecisionTreeClassifier


##indexers = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in features ]
indexer = StringIndexer(inputCol="Symbol", outputCol="Symbol_Index")
indexer2 = StringIndexer(inputCol="Date", outputCol="Date_Index")
assembler3 = VectorAssembler(inputCols=["Symbol_Index","Date_Index"], outputCol="features")
lr = LinearRegression(predictionCol="predictionX")
##kmeans = KMeans(featuresCol="features", predictionCol="prediction2").setK(3).setSeed(1)
##naive = NaiveBayes(smoothing=1.0, modelType="multinomial",featuresCol="features", predictionCol="prediction3")
##dtc = DecisionTreeClassifier(labelCol="Symbol_Index", featuresCol="features", predictionCol="prediction4")
##layers = [4, 5, 4, 3]
##mlpc = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

pipeline = Pipeline(stages=[indexer, indexer2, assembler3, lr])
##pipeline = Pipeline(stages=[indexer, indexer2, assembler3, lr, kmeans, naive])
##pipeline = Pipeline(stages=[indexer, indexer2, assembler3, dtc])
##pipeline = Pipeline(stages=[indexer, indexer2, assembler3, mlpc])

df_r = pipeline.fit(training).transform(training)
df_r.show()

+------+------+----------+------------+----------+----------+------------------+
| label|Symbol|      Date|Symbol_Index|Date_Index|  features|       predictionX|
+------+------+----------+------------+----------+----------+------------------+
| 10.87|     F|2018-04-04|         0.0|      10.0|[0.0,10.0]| 11.21623001091062|
| 11.01|     F|2018-04-03|         0.0|       9.0| [0.0,9.0]|11.216927454469154|
| 11.06|     F|2018-04-02|         0.0|       8.0| [0.0,8.0]|11.217624898027687|
| 11.34|     F|2018-04-05|         0.0|       3.0| [0.0,3.0]|11.221112115820354|
| 11.25|     F|2018-04-09|         0.0|      13.0|[0.0,13.0]| 11.21413768023502|
| 11.28|     F|2018-04-06|         0.0|       5.0| [0.0,5.0]|11.219717228703287|
| 11.38|     F|2018-04-11|         0.0|       7.0| [0.0,7.0]| 11.21832234158622|
| 11.35|     F|2018-04-13|         0.0|       6.0| [0.0,6.0]|11.219019785144754|
| 11.36|     F|2018-04-16|         0.0|       2.0| [0.0,2.0]|11.221809559378887|
| 11.43|     F|2018-04-17|  

In [ ]:
display(df_r)